In [7]:
import pandas as pd
import numpy as np


df = pd.read_csv(r"D:\MSDS CODE\DSLC_EXAM_S25M19030\output\data\cleaned_weather.csv", parse_dates=["DATE"])

#### DESCRIPTIVE ANALYTICS

In [10]:
df[[ "TMP_C",
        "DEW_C",
        "WIND_MS",
        "VIS_M",
        "PRECIP_MM"
    ]].describe()

,TMP_C,DEW_C,WIND_MS,VIS_M,PRECIP_MM
count,8722.000000,8704.000000,8754.000000,8754.000000,8754.000000
mean,24.241034,18.336213,2.738257,11399.431460,2.859070
std,2.910246,1.450463,36.991940,54085.470202,37.437571
min,14.800000,3.100000,0.000000,0.000000,0.000000
25%,22.000000,17.800000,0.000000,6000.000000,0.000000
50%,24.500000,18.500000,1.000000,9000.000000,0.000000
75%,26.500000,19.200000,2.100000,10000.000000,0.000000
max,37.500000,31.500000,999.900000,999999.000000,999.900000


##### Data skewness

In [12]:
dist_stats = pd.DataFrame({
    "skewness": [
        df["TMP_C"].skew(),
        df["PRECIP_MM"].skew()
    ],
    "kurtosis": [
        df["TMP_C"].kurtosis(),
        df["PRECIP_MM"].kurtosis()
    ]
}, index=["TMP_C", "PRECIP_MM"])

In [14]:
dist_stats

,skewness,kurtosis
TMP_C,-0.142917,-0.686513
PRECIP_MM,25.928986,687.191271


#### QUANTILE ANALYTICS

In [16]:
quantiles = df[["TMP_C", "PRECIP_MM"]].quantile(
    [0.90, 0.95, 0.99]
)

quantiles.index = ["P90", "P95", "P99"]
print(quantiles)

     TMP_C  PRECIP_MM
P90   27.8        2.0
P95   28.5       10.0
P99   30.2       32.0


##### EXTREME HEAT - ANALYSIS

In [24]:
#taking 95th percentile
heat_threshold = df["TMP_C"].quantile(0.95)
print("\nExtreme heat threshold (P95):", heat_threshold)


Extreme heat threshold (P95): 28.5


In [26]:
df["heat_extreme"] = (df["TMP_C"] > heat_threshold).astype(int)

print("\nHeat extreme counts:")
print(df["heat_extreme"].value_counts())


Heat extreme counts:
heat_extreme
0    8353
1     401
Name: count, dtype: int64


#### EXTREME RAIN - ANALYSIS

In [29]:
rain_threshold = df["PRECIP_MM"].quantile(0.95)
print("\nExtreme rain threshold (P95):", rain_threshold)

df["rain_extreme"] = (df["PRECIP_MM"] > rain_threshold).astype(int)

print("\nRain extreme counts:")
print(df["rain_extreme"].value_counts())


Extreme rain threshold (P95): 10.0

Rain extreme counts:
rain_extreme
0    8329
1     425
Name: count, dtype: int64


In [33]:
# derived column/feature
df["TEMP_DEW_SPREAD"] = df["TMP_C"] - df["DEW_C"]

#### CONDITIONAL MEANS — HEAT EXTREMES

In [35]:
df.groupby("heat_extreme")[[
    "DEW_C",
    "TEMP_DEW_SPREAD",
    "WIND_MS"
]].mean()


CONDITIONAL MEANS — HEAT EXTREMES



,DEW_C,TEMP_DEW_SPREAD,WIND_MS
heat_extreme,,,
0,18.415862,5.574943,2.778726
1,16.687032,12.973566,1.895262


#### CONDITIONAL MEANS — RAIN EXTREMES

In [44]:
df.groupby("rain_extreme")[[
    "VIS_M",
    "WIND_MS",
    "PRECIP_MM"
]].mean()

,VIS_M,WIND_MS,PRECIP_MM
rain_extreme,,,
0,11544.221875,2.609653,0.377656
1,8561.880000,5.258588,51.488941
